In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv('train.csv')

In [3]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
data.shape

(20800, 5)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
data.dropna(inplace=True)

In [8]:
data.shape

(18285, 5)

In [9]:
data.reset_index(inplace=True)

In [10]:
data['label'].value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [11]:
X = data.drop(['label'],axis=1)

In [12]:
Y=data['label']

In [13]:
import gensim
import nltk
import re

In [14]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kunaljain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(data)):
    review = re.sub('[^a-zA-Z]',' ', data['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
v=corpus[0].split()
print(v)

['hous', 'dem', 'aid', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweet']


In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2000,ngram_range=(1,3))
X_cv = cv.fit_transform(corpus).toarray()

In [19]:
count_df = pd.DataFrame(X_cv, columns=cv.get_feature_names())

/Users/kunaljain/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [20]:
count_df.shape

(18285, 2000)

In [21]:
X_cv.shape

(18285, 2000)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cv, Y, test_size=0.33, random_state=0)

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
model=LogisticRegression()

In [25]:
model.fit(X_train,y_train)

LogisticRegression()

In [26]:
pred = model.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [28]:
accuracy_score(y_test,pred)

0.9294117647058824

In [29]:
confusion_matrix(y_test,pred)

array([[3073,  316],
       [ 110, 2536]])

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=2000,ngram_range=(1,3))
X_tf=tfidf_v.fit_transform(corpus).toarray()

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tf, Y, test_size=0.33, random_state=0)

In [32]:
model.fit(X_train,y_train)

LogisticRegression()

In [33]:
pred1 = model.predict(X_test)

In [34]:
accuracy_score(y_test,pred1)

0.923777961888981

In [35]:
from tensorflow.keras.preprocessing.text import one_hot
import tensorflow as tf

In [36]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [37]:
onehot = [one_hot(word,5000) for word in corpus]

In [38]:
sent_length = 20

In [39]:
model=Sequential()

Metal device set to: Apple M1


2022-08-31 00:49:15.050806: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-31 00:49:15.050897: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [40]:
embedded_docs = pad_sequences(onehot,padding='pre',maxlen=sent_length)

In [41]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4421,
       1586, 2302, 4702, 1309, 2633, 2687, 3756,  442, 3554], dtype=int32)

In [42]:
model.add(Embedding(5000,40,input_length=sent_length))

In [43]:
model.add(Bidirectional(LSTM(100)))

In [44]:
model.add(Dense(1,activation='sigmoid'))

In [45]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [47]:
x_final = np.array(embedded_docs)
y_final = np.array(Y)

In [48]:
x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [49]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=20,batch_size=64)

Epoch 1/20


2022-08-31 00:49:15.412198: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-31 00:49:16.633791: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-31 00:49:16.835376: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-31 00:49:16.845565: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-31 00:49:16.986042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-31 00:49:17.001112: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


192/192 [==============================] - ETA: 0s - loss: 0.3071 - accuracy: 0.8509

2022-08-31 00:49:22.629428: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-31 00:49:22.703192: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-31 00:49:22.710634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


192/192 [==============================] - 8s 35ms/step - loss: 0.3071 - accuracy: 0.8509 - val_loss: 0.2011 - val_accuracy: 0.9155
Epoch 2/20
192/192 [==============================] - 6s 30ms/step - loss: 0.1370 - accuracy: 0.9469 - val_loss: 0.2244 - val_accuracy: 0.9167
Epoch 3/20
192/192 [==============================] - 6s 29ms/step - loss: 0.0912 - accuracy: 0.9673 - val_loss: 0.2283 - val_accuracy: 0.9143
Epoch 4/20
192/192 [==============================] - 6s 29ms/step - loss: 0.0585 - accuracy: 0.9811 - val_loss: 0.2693 - val_accuracy: 0.9152
Epoch 5/20
192/192 [==============================] - 6s 30ms/step - loss: 0.0311 - accuracy: 0.9899 - val_loss: 0.4074 - val_accuracy: 0.9133
Epoch 6/20
192/192 [==============================] - 6s 29ms/step - loss: 0.0189 - accuracy: 0.9939 - val_loss: 0.4348 - val_accuracy: 0.9138
Epoch 7/20
192/192 [==============================] - 6s 29ms/step - loss: 0.0143 - accuracy: 0.9966 - val_loss: 0.5011 - val_accuracy: 0.9115
Epoch 8/20

In [50]:
pred3 = model.predict(x_test)

  6/189 [..............................] - ETA: 1s  

2022-08-31 00:51:11.405555: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-31 00:51:11.471237: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-31 00:51:11.481600: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


189/189 [==============================] - 2s 10ms/step


In [51]:
pred3 = np.where(pred3>0.5, 1,0)

In [52]:
accuracy_score(y_test,pred3)

0.9115161557580779

In [53]:
confusion_matrix(y_test,pred3)

array([[3120,  299],
       [ 235, 2381]])